# This Notebook is for reviewing Cleaned EEG files and making edits to them

## Imports and Config settings

In [ ]:
import os
from pathlib import Path

import pandas as pd
import mne

from typing import Tuple

In [ ]:
mne.viz.set_browser_backend("qt")
mne.set_log_level("WARNING")

In [ ]:
from typing import TypeAlias

FileName: TypeAlias = str | bytes | os.PathLike

## Load (Or Create) a Tracking Sheet

In [ ]:
QCR_FNAME = Path.cwd() / "LISTEN-PHONEMES-EEG-QCR-Log.csv"
RECOMPUTE = False

deriv_dir = Path("/Volumes/UBUNTU18/USC/listen/derivatives/pylossless/run-01")
assert deriv_dir.exists()

QCR_DIR = Path("/Volumes/UBUNTU18/USC/listen/derivatives/QCR/run-01")
QCR_DIR.mkdir(exist_ok=True, parents=True)

sub_dirs = [sub_dir for sub_dir in deriv_dir.iterdir() if sub_dir.is_dir()]

if not QCR_FNAME.exists() or RECOMPUTE:
    filepaths = sub_dirs
    filenames = [fpath.name for fpath in filepaths]
    df = pd.DataFrame({"file" : filenames, "filepath": filepaths})
    df["status"] = pd.NA
    df["reviewer"] = pd.NA
    df["exclude"] = pd.NA
    df["notes"] = pd.NA
    df.to_csv(QCR_FNAME, index=False)
# Load the sub_dirs from the CSV file
qcr_df = pd.read_csv(QCR_FNAME)
qcr_df.head()

In [ ]:
def qcr_iterator(qcr_df):
    for idx, row in qcr_df.iterrows():
        if pd.isnull(row["status"]):
            return row

In [ ]:
next_up: pd.DataFrame = qcr_iterator(qcr_df)
next_up

## Load the EEG Files

In [ ]:
def get_derivatives_fpaths(fpath: FileName) -> dict[str: FileName]:
    fpath = Path(fpath)
    fpaths_dict: dict = {}
    
    fpath_raw = list(fpath.glob("*_proc-cleaned_raw.fif"))
    fpath_ica = list(fpath.glob("*_proc-ica.fif"))
    fpath_iclabels = list(fpath.glob("*_iclabels.csv"))
    assert len(fpath_raw) == 1
    assert len(fpath_ica) == 1
    assert len(fpath_iclabels) == 1
    return {"raw": fpath_raw[0], "ica": fpath_ica[0], "ic_labels": fpath_iclabels[0]}

In [ ]:
fpaths = get_derivatives_fpaths(next_up["filepath"])
fpaths

In [ ]:
def load_derivatives(fpath_dict: dict[str, FileName]) -> Tuple[mne.io.BaseRaw, mne.preprocessing.ICA, pd.DataFrame]:
    raw = mne.io.read_raw_fif(fpath_dict["raw"])
    ica = mne.preprocessing.ica.read_ica(fpath_dict["ica"])
    ic_labels = pd.read_csv(fpath_dict["ic_labels"])
    return raw, ica, ic_labels

In [ ]:
raw, ica, ic_labels = load_derivatives(fpaths)
raw

## Review the EEG data

In [ ]:
fig = raw.plot_sensors(show_names=True, show=False)

In [ ]:
raw.plot(theme="light")

In [ ]:
epochs = mne.make_fixed_length_epochs(raw, 2, preload=True)
psd = epochs.set_eeg_reference("average").compute_psd(fmin=2, fmax=50)
del epochs
fig = psd.plot(show=False)

## Update the tracking sheet with your notes about the Data

In [ ]:
def update_qcr(
    row: pd.Series,
    *,
    status: str,
    notes: str,
    exclude: str,
    reviewer: str = "SH",    
) -> pd.DataFrame:
    row_idx = row.name
    qcr_df.astype({"status": "object", "notes": "object"})
    qcr_df.at[row_idx, "status"] = status
    qcr_df.at[row_idx, "notes"] = notes
    qcr_df.at[row_idx, "exclude"] = exclude
    qcr_df.at[row_idx, "reviewer"] = reviewer
    qcr_df.to_csv(QCR_FNAME, index=False)
    return pd.read_csv(QCR_FNAME)

In [ ]:
qcr_df = update_qcr(
    next_up,
    status="Done.",
    notes="Mostly bad. Except very end",
    exclude=False,
)
qcr_df.head()

## Save the edited EEG file

In [ ]:
def save_qcr_raw(raw: mne.io.BaseRaw) -> FileName:
    fpath: FileName = raw.filenames[0]
    sub_dir = fpath.parent.name
    fname = fpath.name
    out_dir = QCR_DIR / sub_dir
    assert QCR_DIR.exists()
    out_dir.mkdir(exist_ok=True)
    out_fpath = out_dir / fname
    raw.save(out_fpath)
    return out_fpath

In [ ]:
save_qcr_raw(raw)